# 自动微分

作为一个演示例子，假设我们想对函数y=2x<sup>T</sup>x关于列向量x求导。 首先，我们创建变量x并为其分配一个初始值。

In [2]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算y关于x的梯度之前，需要一个地方来存储梯度。

重要的是，我们不会在每次对一个参数求导时都分配新的内存。

因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能
很快就会将内存耗尽。

注意，一个标量函数关于向量x的梯度是向量，并且与x具有相同的形状

In [3]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad  # 默认值是None

In [4]:
print(x.grad)

None


In [5]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [6]:
help(y.backward)

Help on method backward in module torch._tensor:

backward(gradient=None, retain_graph=None, create_graph=False, inputs=None) method of torch.Tensor instance
    Computes the gradient of current tensor wrt graph leaves.
    
    The graph is differentiated using the chain rule. If the tensor is
    non-scalar (i.e. its data has more than one element) and requires
    gradient, the function additionally requires specifying a ``gradient``.
    It should be a tensor of matching type and shape, that represents
    the gradient of the differentiated function w.r.t. ``self``.
    
    This function accumulates gradients in the leaves - you might need to zero
    ``.grad`` attributes or set them to ``None`` before calling it.
    See :ref:`Default gradient layouts<default-grad-layouts>`
    for details on the memory layout of accumulated gradients.
    
    .. note::
    
        If you run any forward ops, create ``gradient``, and/or call ``backward``
        in a user-specified CUDA stream 

In [7]:
print(y.backward)

<bound method Tensor.backward of tensor(28., grad_fn=<MulBackward0>)>


In [8]:
x.grad  # y对x的梯度
print(x.grad)

None


In [9]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [10]:
 # 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

In [11]:
y

tensor(6., grad_fn=<SumBackward0>)


#### 非标量变量的反向传播

In [12]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

### 分离计算

In [13]:
x.grad.zero_()
y = x * x
y

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)

通过detach分离 ，使u变成常量

In [14]:
u = y.detach()  # 分离计算，不再需要梯度跟踪
u

tensor([0., 1., 4., 9.])

In [15]:
z = u * x
z

tensor([ 0.,  1.,  8., 27.], grad_fn=<MulBackward0>)

所以梯度也就是偏导变成了u

In [16]:
z.sum()

tensor(36., grad_fn=<SumBackward0>)

In [17]:
z.sum().backward()

In [18]:
x.grad  # x的梯度没有变化

tensor([0., 1., 4., 9.])

In [30]:
x.grad == u

tensor([True, True, True, True])

In [31]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

Python控制流的梯度计算

In [ ]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [25]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [26]:
a.grad == d / a

tensor(True)